In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import pyodbc

In [2]:
connection = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
                      'Server=DESKTOP-ATL660H;'
                      'Database=AdventureWorks2012;'
                      'Trusted_Connection=yes;')


In [3]:
query ='''
SELECT 
C.SalesOrderID
,A.Name
,B.OrderQty
,D.Name as Region
FROM[Production].[Product]  AS A
LEFT JOIN [Sales].[SalesOrderDetail] AS B
ON A.ProductID = B.ProductID
LEFT JOIN [Sales].[SalesOrderHeader] AS C
ON B.SalesOrderID = C.SalesOrderID
LEFT JOIN [Sales].[SalesTerritory] AS D
ON C.[TerritoryID] = D.[TerritoryID]
WHERE C.OnlineOrderFlag = 1
'''


In [4]:
data= pd.read_sql(query,connection)
data.head()

,SalesOrderID,Name,OrderQty,Region
0,43697,"Road-150 Red, 62",1,Canada
1,43698,"Mountain-100 Silver, 44",1,France
2,43699,"Mountain-100 Silver, 44",1,Northwest
3,43700,"Road-650 Black, 62",1,Southwest
4,43701,"Mountain-100 Silver, 44",1,Australia


In [5]:
data.dtypes

SalesOrderID     int64
Name            object
OrderQty         int64
Region          object
dtype: object

In [6]:
#stripping of the white spaces form the Name
data['Name'] = data['Name'].str.strip()

#dropping rows with missing values in salesorderid
# converting  salesorderid to type string for the algorithm to understand it as astring data type
data.dropna(axis=0, subset=['SalesOrderID'], inplace= True)
data['SalesOrderID'] = data['SalesOrderID'].astype('str')

In [7]:
data.head()

,SalesOrderID,Name,OrderQty,Region
0,43697,"Road-150 Red, 62",1,Canada
1,43698,"Mountain-100 Silver, 44",1,France
2,43699,"Mountain-100 Silver, 44",1,Northwest
3,43700,"Road-650 Black, 62",1,Southwest
4,43701,"Mountain-100 Silver, 44",1,Australia


In [8]:
data['Region'].value_counts()

Australia         13345
Southwest         12265
Northwest          8993
Canada             7620
United Kingdom     6906
Germany            5625
France             5558
Southeast            39
Northeast            27
Central              20
Name: Region, dtype: int64

In [9]:
#separating transaction by region 
basket = (data
            .groupby(['SalesOrderID', 'Name'])['OrderQty']
            .sum().unstack().reset_index().fillna(0)
            .set_index('SalesOrderID'))


In [10]:
# applying one hot encoding to converting all values to 0's and 1's

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)


In [11]:
basket_sets

Name,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,HL Mountain Tire,HL Road Tire,"Half-Finger Gloves, L",...,"Touring-3000 Blue, 62","Touring-3000 Yellow, 44","Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S"
SalesOrderID,,,,,,,,,,,,,,,,,,,,,
43697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43700,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43702,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43703,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43704,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# generating frequent item sets
frequent_itemsets = apriori(basket_sets, min_support=0.04, use_colnames=True)

In [13]:
# viewing frequent itemset
frequent_itemsets

,support,itemsets
0,0.079179,(AWC Logo Cap)
1,0.076684,(Fender Set - Mountain)
2,0.050472,(HL Mountain Tire)
3,0.041975,(ML Mountain Tire)
4,0.073213,(Mountain Bottle Cage)
5,0.111898,(Mountain Tire Tube)
6,0.115369,(Patch Kit/8 Patches)
7,0.061897,(Road Bottle Cage)
8,0.085903,(Road Tire Tube)
9,0.075382,"(Sport-100 Helmet, Black)"


In [14]:
# creating rules  
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [15]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Water Bottle - 30 oz.),(Mountain Bottle Cage),0.153440,0.073213,0.061174,0.398680,5.445483,0.049940,1.541255
1,(Mountain Bottle Cage),(Water Bottle - 30 oz.),0.073213,0.153440,0.061174,0.835556,5.445483,0.049940,5.147999
2,(Road Bottle Cage),(Water Bottle - 30 oz.),0.061897,0.153440,0.054991,0.888435,5.790107,0.045494,7.588013
3,(Water Bottle - 30 oz.),(Road Bottle Cage),0.153440,0.061897,0.054991,0.358388,5.790107,0.045494,1.462104
